In [1]:
from scipy import *
from matplotlib.pyplot import *
from scipy.linalg import *
from numpy.linalg import *
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import splu
from timeit import default_timer as timer

# T2

In [2]:
def lufact(A):
	"""
	lufact(A)

	Compute the LU factorization of square matrix A, returning the factors.
	"""
	n = A.shape[0]
	L = np.eye(n)
	U = A.copy()

	# Gaussian elimination
	for j in range(n-1):
		for i in range(j+1,n):
			L[i,j] = U[i,j] / U[j,j]   # row multiplier
			U[i,j:] = U[i,j:] - L[i,j]*U[j,j:]
	return L, np.triu(U)

def forwardsub(L,b):
	"""
 	forwardsub(L,b)

	Solve the lower-triangular linear system with matrix L and right-hand side
	vector b.
	"""
	n = len(b)
	x = np.zeros(n)
	for i in range(n):
		s = L[i,:i] @ x[:i]
		x[i] = ( b[i] - s ) / L[i,i]
	return x


def backsub(U,b):
	"""
	backsub(U,b)

	Solve the upper-triangular linear system with matrix U and right-hand side
	vector b.
	"""
	n = len(b)
	x = np.zeros(n)
	for i in range(n-1,-1,-1):
		s = U[i,i+1:] @ x[i+1:]
		x[i] = ( b[i] - s ) / U[i,i]
	return x


    

def matrix_T(x, y):
    T = np.array([[1,0,0], [0, 1, 0], [x, y, 1]])
    return T

def matrix_R(theta):
    R = np.array([[np.cos(theta), np.sin(theta), 0],
                  [-np.sin(theta), np.cos(theta), 0],
                 [0, 0, 1]])
    return R

In [3]:
A = matrix_T(3, -1)@matrix_R(np.pi/5)@matrix_T(-3, 1)
z = np.array([2, 2, 1])

In [4]:
A

array([[ 0.80901699,  0.58778525,  0.        ],
       [-0.58778525,  0.80901699,  0.        ],
       [ 0.01483624,  1.95433876,  1.        ]])

In [5]:
## T2 (a)
b = A@z
print("b = ", b)

b =  [2.79360449 0.44246348 4.93835   ]


In [6]:
## T2 (b)
L,U = lufact(A)
print("L=",L)
print("U=",U)

L= [[ 1.          0.          0.        ]
 [-0.72654253  1.          0.        ]
 [ 0.0183386   1.57237275  1.        ]]
U= [[0.80901699 0.58778525 0.        ]
 [0.         1.23606798 0.        ]
 [0.         0.         1.        ]]


In [7]:
## T2 (b)
y = forwardsub(L,b)
x = backsub(U,y)
print("x - z = ", x-z)

x - z =  [ 0.0000000e+00  0.0000000e+00 -4.4408921e-16]


# T3

In [8]:
## T3 (a)
A = np.array([[1,0,0,0,1e12],[1,1,0,0,0], [0,1,1,0,0],[0,0,1,1,0], [0,0,0,1,0]])
x_hat = np.array([0, 1/3, 2/3, 1, 4/3])
b = A @ x_hat
L,U = lufact(A)
y = forwardsub(L,b)
x = backsub(U,y)
print("x", x)
print("x - x_hat = ", x-x_hat)

x [0.         0.33325195 0.66674805 1.         1.33333333]
x - x_hat =  [ 0.00000000e+00 -8.13802083e-05  8.13802083e-05  0.00000000e+00
  0.00000000e+00]


In [9]:
## T3 (b)
A[0, 4] = 1e20
b = A @ x_hat
L,U = lufact(A)
y = forwardsub(L,b)
x = backsub(U,y)
print("x", x)
print("x - x_hat = ", x-x_hat)

x [0.         0.         0.         0.         1.33333333]
x - x_hat =  [ 0.         -0.33333333 -0.66666667 -1.          0.        ]


# T4

In [10]:
## T4
A = np.array([[1,1,0,1,0,0], [0,1,1,0,1,0], [0,0,1,1,0,1], [1,0,0,1,1,0], [1,1,0,0,1,1], [0,1,1,0,0,1]])
L, U = lufact(A)
L_inv = inv(L)
U_inv = inv(U)
print("L", L)
print("U", U)
print("inverse of L", L_inv)
print("inverse of U", U_inv)

L [[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [ 1. -1.  1.  1.  0.  0.]
 [ 1.  0.  0.  1.  1.  0.]
 [ 0.  1.  0. -0.  1.  1.]]
U [[ 1  1  0  1  0  0]
 [ 0  1  1  0  1  0]
 [ 0  0  1  1  0  1]
 [ 0  0  0 -1  2 -1]
 [ 0  0  0  0 -1  2]
 [ 0  0  0  0  0 -1]]
inverse of L [[ 1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.]
 [-1.  1. -1.  1.  0.  0.]
 [ 0. -1.  1. -1.  1.  0.]
 [ 0.  0. -1.  1. -1.  1.]]
inverse of U [[ 1. -1.  1.  2.  3.  5.]
 [ 0.  1. -1. -1. -1. -2.]
 [ 0.  0.  1.  1.  2.  4.]
 [-0. -0. -0. -1. -2. -3.]
 [-0. -0. -0. -0. -1. -2.]
 [-0. -0. -0. -0. -0. -1.]]


# T5

In [11]:
## T5
def lufact2(A):
    B = A.T
    L, U = lufact(B)
    return U.T, L.T

In [12]:
A = np.array([[6,-2,-4,4], [3, -3, -6, 1], [-12, 8, 21, -8], [-6, 0, -10, 7]])

In [13]:
L, U = lufact(A)
L_new, U_new = lufact2(A)
print("A = ", A)
print("L = ", L)
print("U = ", U)
print("Verification: LU = ", L@U)
print("L_new = ", L_new)
print("U_new = ", U_new)
print("Verification: L_new U_new = ", L_new@U_new)

A =  [[  6  -2  -4   4]
 [  3  -3  -6   1]
 [-12   8  21  -8]
 [ -6   0 -10   7]]
L =  [[ 1.   0.   0.   0. ]
 [ 0.5  1.   0.   0. ]
 [-2.  -2.   1.   0. ]
 [-1.   1.  -2.   1. ]]
U =  [[ 6 -2 -4  4]
 [ 0 -2 -4 -1]
 [ 0  0  5 -2]
 [ 0  0  0  8]]
Verification: LU =  [[  6.  -2.  -4.   4.]
 [  3.  -3.  -6.   1.]
 [-12.   8.  21.  -8.]
 [ -6.   0. -10.   7.]]
L_new =  [[  6   0   0   0]
 [  3  -2   0   0]
 [-12   4   5   0]
 [ -6  -2 -10   8]]
U_new =  [[ 1.         -0.33333333 -0.66666667  0.66666667]
 [ 0.          1.          2.          0.5       ]
 [ 0.          0.          1.         -0.4       ]
 [ 0.          0.          0.          1.        ]]
Verification: L_new U_new =  [[  6.  -2.  -4.   4.]
 [  3.  -3.  -6.   1.]
 [-12.   8.  21.  -8.]
 [ -6.   0. -10.   7.]]


# T6 (b)

In [14]:
def determinant(A):
    L, U = lufact(A)
    n = A.shape[0]

    det_value = 1.0
    for i in range(n):
        det_value = det_value * U[i, i]
    return det_value

In [15]:
A = np.random.rand(5, 5)

In [16]:
print("determinant(A) = ", determinant(A))
print("det(A) = ", det(A))
print("determinant(A) - det(A) = ",  determinant(A) - det(A))

determinant(A) =  -0.007406532792237683
det(A) =  -0.007406532792237033
determinant(A) - det(A) =  -6.496539417533143e-16
